In [5]:
import simpleaudio as sa
import numpy as np
from qiskit import *
from qiskit.visualization import plot_bloch_multivector, circuit_drawer
from PIL import ImageTk, Image
pi = np.pi
import tkinter
from tkinter import *
import qutip
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

def sin(f,t):
    return np.sin(f*t*2*pi)
    return sound

def square(f,t):
    n = 10
    sound = 0
    for i in range(1,int((n+1)/2)):
        sound += 4*np.sin((2*i-1)*f*t*2*pi)/((2*i-1)*pi)
    return sound

def letter(phi):
    note_list = ['C','Db','D','Eb','E','F','Gb','G','Ab','A','Bb','B']
    return note_list[int(round((6/pi)*phi)) % 12]

def sound2(circ): 
    
    backend = BasicAer.get_backend('statevector_simulator')
    result = execute(circ, backend).result()
    sv = result.get_statevector()
    r = []
    phi = []
    for i in range(4):
        r.append(abs(sv[i]))
        p = np.real(np.angle(sv[i]))
        if p < 0:
            p = 2*pi + p
        phi.append(p)
    
    # Swap
    r2 = r[2]
    r[2] = r[1]
    r[1] = r2
    
    phi2 = phi[2]
    phi[2] = phi[1]
    phi[1] = phi2
    
    f = 440*2**(-9/12)  # Our played note will be 440 Hz
    fs = 44100  # 44100 samples per second
    seconds = 2  # Note duration of 2 seconds
    t = np.linspace(0, seconds, seconds * fs, False) # Generate array with seconds*sample_rate steps

    # Create sound
    left =  r[0]**2*sin(f,t)+r[0]**2*square(f,t)+r[1]**2*sin(2**(phi[1]/(2*pi))*f,t)+r[2]**2*square(2**(phi[2]/(2*pi))*f,t)
    right = r[1]**2*square(2**(phi[1]/(2*pi))*f,t)+r[2]**2*sin(2**(phi[2]/(2*pi))*f,t)+r[3]**2*sin(2**(phi[3]/(2*pi))*f,t)+r[3]**2*square(2**(phi[3]/(2*pi))*f,t)
    
    note = np.dstack((left,right))[0]

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))
    audio = audio.astype(np.int16) # Convert to 16-bit data  
    
    # Playback
    play_obj = sa.play_buffer(audio, 2, 2, fs)
    play_obj.wait_done()
    

# Create window

root = Tk()
root.title('Mubits')
root.geometry("750x350")

head = Text(root)
head.insert(INSERT, 'Two Qubits')
head.place(x = 300,y = 5,height = 30,width = 80)

text = Text(root)
text.insert(INSERT, "|q0> ------------------------------")
text.place(x = 20,y = 30, width = 250, height = 25)

text = Text(root)
text.insert(INSERT, "|q1> ------------------------------")
text.place(x = 20,y = 60, width = 250, height = 25)

entry00 = Entry(root, width=3)
entry00.place(x=70,y=30)

entry10 = Entry(root, width=3)
entry10.place(x=70,y=60)

entry01 = Entry(root, width=3)
entry01.place(x=120,y=30)

entry11 = Entry(root, width=3)
entry11.place(x=120,y=60)

entry02 = Entry(root, width=3)
entry02.place(x=170,y=30)

entry12 = Entry(root, width=3)
entry12.place(x=170,y=60)

entry03 = Entry(root, width=3)
entry03.place(x=220,y=30)

entry13 = Entry(root, width=3)
entry13.place(x=220,y=60)

# Initialize text
text = Text(root)
text.insert(INSERT, '|q_0q_1> = 1.0 C |LL> + 0.0 C |LR> + 0.0 C |RL> + 0.0 C |RR> \n Instrument 1: Left = 1.0 C and 0.0 C Right = 0.0 C and 0.0 C \n Instrument 2: Left = 1.0 C and 0.0 C Right = 0.0 C and 0.0 C')
text.place(x = 300,y = 30,height = 100,width = 500)

# Initialize Bloch sphere
q = QuantumRegister(2, 'q')
circ = QuantumCircuit(q)

backend = BasicAer.get_backend('statevector_simulator')
result = execute(circ, backend).result()
sv = result.get_statevector()
fig = plot_bloch_multivector(sv)

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().place(x = 50,y = 100)


def run():

    q = QuantumRegister(2, 'q')
    circ = QuantumCircuit(q)
    
    if entry00.get() == 'X' and entry10.get() != 'C':
        circ.x(q[0])
    if entry10.get() == 'X' and entry00.get() != 'C':
        circ.x(q[1])
    if entry00.get() == 'X' and entry10.get() == 'C':
        circ.cx(q[1],q[0])
    if entry10.get() == 'X' and entry00.get() == 'C':
        circ.cx(q[0],q[1])
    if entry00.get() == 'H':
        circ.h(q[0])
    if entry10.get() == 'H':
        circ.h(q[1])
    if entry00.get().startswith('Ry'):
        theta = float(entry00.get()[2:])*pi
        circ.ry(theta,q[0])
    if entry10.get().startswith('Ry'):
        theta = float(entry10.get()[2:])*pi
        circ.ry(theta,q[1])  
    if entry00.get().startswith('r') and entry10.get() != 'C':
        phi = (int(entry00.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[0])
    if entry10.get().startswith('r') and entry00.get() != 'C':
        phi= (int(entry10.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[1])
    if entry10.get().startswith('r') and entry00.get() == 'C':
        phi = (int(entry10.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[0],q[1])
    if entry00.get().startswith('r') and entry10.get() == 'C':
        phi = (int(entry00.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[1],q[0])
    if entry00.get() == 'P' and entry10.get() == 'P':
        circ.cx(q[1],q[0])
        circ.cry(-pi/2,q[0],q[1])
        circ.cx(q[1],q[0])
        
    if entry01.get() == 'X' and entry11.get() != 'C':
        circ.x(q[0])
    if entry11.get() == 'X' and entry01.get() != 'C':
        circ.x(q[1])
    if entry01.get() == 'X' and entry11.get() == 'C':
        circ.cx(q[1],q[0])
    if entry11.get() == 'X' and entry01.get() == 'C':
        circ.cx(q[0],q[1])
    if entry01.get() == 'H':
        circ.h(q[0])
    if entry11.get() == 'H':
        circ.h(q[1])
    if entry01.get().startswith('Ry'):
        theta = float(entry01.get()[2:])*pi
        circ.ry(theta,q[0])
    if entry11.get().startswith('Ry'):
        theta = float(entry11.get()[2:])*pi
        circ.ry(theta,q[1])  
    if entry01.get().startswith('r') and entry11.get() != 'C':
        phi = (int(entry01.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[0])
    if entry11.get().startswith('r') and entry01.get() != 'C':
        phi= (int(entry11.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[1])
    if entry11.get().startswith('r') and entry01.get() == 'C':
        phi = (int(entry11.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[0],q[1])
    if entry01.get().startswith('r') and entry11.get() == 'C':
        phi = (int(entry01.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[1],q[0])
    if entry01.get() == 'P' and entry11.get() == 'P':
        circ.cx(q[1],q[0])
        circ.cry(-pi/2,q[0],q[1])
        circ.cx(q[1],q[0])
        
    if entry02.get() == 'X' and entry12.get() != 'C':
        circ.x(q[0])
    if entry12.get() == 'X' and entry02.get() != 'C':
        circ.x(q[1])
    if entry02.get() == 'X' and entry12.get() == 'C':
        circ.cx(q[1],q[0])
    if entry12.get() == 'X' and entry02.get() == 'C':
        circ.cx(q[0],q[1])
    if entry02.get() == 'H':
        circ.h(q[0])
    if entry12.get() == 'H':
        circ.h(q[1])
    if entry02.get().startswith('Ry'):
        theta = float(entry02.get()[2:])*pi
        circ.ry(theta,q[0])
    if entry12.get().startswith('Ry'):
        theta = float(entry12.get()[2:])*pi
        circ.ry(theta,q[1])  
    if entry02.get().startswith('r') and entry12.get() != 'C':
        phi = (int(entry02.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[0])
    if entry12.get().startswith('r') and entry02.get() != 'C':
        phi = (int(entry12.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[1])
    if entry12.get().startswith('r') and entry02.get() == 'C':
        phi = (int(entry12.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[0],q[1])
    if entry02.get().startswith('r') and entry12.get() == 'C':
        phi = (int(entry02.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[1],q[0])  
    if entry02.get() == 'P' and entry12.get() == 'P':
        circ.cx(q[1],q[0])
        circ.cry(-pi/2,q[0],q[1])
        circ.cx(q[1],q[0])
        
    if entry03.get() == 'X' and entry13.get() != 'C':
        circ.x(q[0])
    if entry13.get() == 'X' and entry03.get() != 'C':
        circ.x(q[1])
    if entry03.get() == 'X' and entry13.get() == 'C':
        circ.cx(q[1],q[0])
    if entry13.get() == 'X' and entry03.get() == 'C':
        circ.cx(q[0],q[1])
    if entry03.get() == 'H':
        circ.h(q[0])
    if entry13.get() == 'H':
        circ.h(q[1])
    if entry03.get().startswith('Ry'):
        theta = float(entry03.get()[2:])*pi
        circ.ry(theta,q[0])
    if entry13.get().startswith('Ry'):
        theta = float(entry13.get()[2:])*pi
        circ.ry(theta,q[1])  
    if entry03.get().startswith('r') and entry13.get() != 'C':
        phi = (int(entry03.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[0])
    if entry13.get().startswith('r') and entry03.get() != 'C':
        phi = (int(entry13.get()[1:])/12)*(2*pi)
        circ.u1(phi,q[1])
    if entry13.get().startswith('r') and entry03.get() == 'C':
        phi = (int(entry13.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[0],q[1])
    if entry03.get().startswith('r') and entry13.get() == 'C':
        phi = (int(entry03.get()[1:])/12)*(2*pi)
        circ.cu1(phi,q[1],q[0])
    if entry03.get() == 'P' and entry13.get() == 'P':
        circ.cx(q[1],q[0])
        circ.cry(-pi/2,q[0],q[1])
        circ.cx(q[1],q[0])
      
    # Get statevector
    backend = BasicAer.get_backend('statevector_simulator')
    result = execute(circ, backend).result()
    sv = result.get_statevector()
        
    # Get coefficients and angles
    r = []
    phi = []
    for i in range(4):
        r.append(abs(sv[i]))
        p = np.real(np.angle(sv[i]))
        if p < 0:
            p = 2*pi + p
        phi.append(p)
    
    # Swap to IBM convention
    r2 = r[2]
    r[2] = r[1]
    r[1] = r2
    
    phi2 = phi[2]
    phi[2] = phi[1]
    phi[1] = phi2

    # Update text
    state = '|q_0q_1> = ' + str(round(r[0],3)) +  letter(phi[0]) + '|LL> +' + str(round(r[1],3)) + letter(phi[1]) + '|LR> +' + str(round(r[2],3)) + letter(phi[2]) + '|RL> +' + str(round(r[3],3)) + letter(phi[3]) + '|RR>' + '\n'
    I1 = 'Instrument 1:' + 'Left = ' + str(round(r[0]**2,3)) + letter(phi[0]) + ' and ' + str(round(r[1]**2,3)) + letter(phi[1]) + ' Right = ' + str(round(r[2]**2,3)) + letter(phi[2]) + ' and ' + str(round(r[3]**2,3)) + letter(phi[3]) + '\n'
    I2 = 'Instrument 2:' + 'Left = ' + str(round(r[0]**2,3)) + letter(phi[0]) + ' and ' + str(round(r[2]**2,3)) + letter(phi[2]) + ' Right = ' + str(round(r[1]**2,3)) + letter(phi[1]) + ' and ' + str(round(r[3]**2,3)) + letter(phi[3])
    text.delete(1.0,END)
    text.insert(INSERT, state+I1+I2)  
        
    # Update Bloch sphere
    fig = plot_bloch_multivector(sv)
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().place(x = 50,y = 100)
    
    # Play corresponding sound
    sound2(circ)

Button(root, text='Run', command=run).place(x=20,y=100)

root.lift()
root.attributes('-topmost',True)
root.mainloop()